# Notebook: pyolimp - Precompensation Examples

In [ ]:
from typing import Callable
from torch import Tensor
from olimp.processing import scale_value, fft_conv
from olimp.precompensation._demo import demo
from olimp.precompensation._demo_cvd import demo as demo_cvd

* Huang

In [ ]:
from olimp.precompensation.basic.huang import huang


def demo_huang(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    ret = huang(image, psf)
    progress(1.0)
    return scale_value(ret, min_val=0, max_val=1.0)


demo("Huang", demo_huang, mono=False)

* Feng Xu

In [ ]:
from olimp.precompensation.analytics.feng_xu import feng_xu


def demo_huang(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    ret = feng_xu(image, psf, lambda_val=2)
    progress(0.8)
    ret = fft_conv(scale_value(ret), psf)
    progress(1.0)
    return ret


demo("Feng Xu", demo_huang, mono=True)

* Achromatic Daltonization

In [ ]:
from olimp.precompensation.optimization.achromatic_daltonization import (
    achromatic_daltonization,
    ColorBlindnessDistortion,
    ADParameters,
    M1Loss,
)
import warnings


def demo_achromatic_daltonization(
    image: Tensor,
    distortion: ColorBlindnessDistortion,
    progress: Callable[[float], None],
) -> tuple[Tensor]:

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return (
            achromatic_daltonization(
                image,
                distortion,
                ADParameters(progress=progress, loss_func=M1Loss()),
            ),
        )


distortion = ColorBlindnessDistortion.from_type("protan")
demo_cvd(
    "Achromatic Daltonization",
    demo_achromatic_daltonization,
    distortion=distortion,
)

* Bregman Jumbo

In [ ]:
from olimp.precompensation.optimization.bregman_jumbo import (
    bregman_jumbo,
    BregmanJumboParameters,
)


def demo_bregman_jumbo(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    return bregman_jumbo(image, psf, BregmanJumboParameters(progress=progress))


demo("Bregman Jumbo", demo_bregman_jumbo)

* CVD Direct Optimization

In [ ]:
from olimp.precompensation.optimization.cvd_direct_optimization import (
    cvd_direct_optimization,
    CVDParameters,
)
from olimp.simulate.color_blindness_distortion import ColorBlindnessDistortion


def demo_cvd_direct_optimization(
    image: Tensor,
    distortion: ColorBlindnessDistortion,
    progress: Callable[[float], None],
) -> Tensor:
    from olimp.evaluation.loss.rms import RMS

    return (
        cvd_direct_optimization(
            image,
            distortion,
            CVDParameters(progress=progress, loss_func=RMS("lab")),
        ),
    )


distortion = ColorBlindnessDistortion.from_type("protan")
demo_cvd(
    "CVD DIRECT OPTIMIZATION",
    demo_cvd_direct_optimization,
    distortion=distortion,
)

* Global Tone Mapping

In [ ]:
from olimp.precompensation.optimization.global_tone_mapping import (
    precompensation_global_tone_mapping,
    GTMParameters,
)


def demo_global_tone_mapping(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    return precompensation_global_tone_mapping(
        image, psf, GTMParameters(progress=progress, lr=0.05)
    )


demo("Global Tone Mapping", demo_global_tone_mapping, mono=False)

* Half-Quadratic

In [ ]:
from olimp.precompensation.optimization.hqs import hqs, HQSParameters


def demo_hqs(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    return hqs(image, psf, HQSParameters(progress=progress))


demo("Half-Quadratic", demo_hqs)

* Ji

In [ ]:
from olimp.precompensation.optimization.ji import ji, JiParameters
from olimp.evaluation.loss.piq import MultiScaleSSIMLoss


def demo_ji(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    return ji(
        image,
        psf,
        JiParameters(
            progress=progress, alpha=1, loss_func=MultiScaleSSIMLoss()
        ),
    )


demo("Ji", demo_ji)

* Montalto (FISTA)

In [ ]:
from olimp.precompensation.optimization.montalto_fista import (
    montalto as montalto_fista,
    MontaltoParameters as FistaMontaltoParameters,
)


def demo_montalto(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    return montalto_fista(
        image, psf, FistaMontaltoParameters(progress=progress)
    )


demo("Montalto (FISTA)", demo_montalto, mono=False)

* Montalto

In [ ]:
from olimp.precompensation.optimization.montalto import (
    montalto as montalto,
    MontaltoParameters,
)


def demo_montalto(
    image: Tensor,
    psf: Tensor,
    progress: Callable[[float], None],
) -> Tensor:
    return montalto(image, psf, MontaltoParameters(progress=progress))


demo("Montalto", demo_montalto)

* Tennenholtz-Zachevsky

In [ ]:
from olimp.precompensation.optimization.tennenholtz_zachevsky import (
    tennenholtz_zachevsky,
    TennenholtzZachevskyParameters,
)


def demo_tennenholtz_zachevsky(
    image: Tensor,
    distortion: ColorBlindnessDistortion,
    progress: Callable[[float], None],
) -> tuple[Tensor]:
    parameters = TennenholtzZachevskyParameters(progress=progress)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return (tennenholtz_zachevsky(image[0], distortion, parameters)[None],)


distortion = ColorBlindnessDistortion.from_type("protan")
demo_cvd(
    "Tennenholtz-Zachevsky",
    demo_tennenholtz_zachevsky,
    distortion=distortion,
)